## MediPal -- LLM testing and selection

### In this section, I set five tasks to test three LLMs' performance, so that I can understand them and design the application properly.
##### Tasks:
1. Reasoning
2. Generate questions based on given content
3. Answer question based on given content
4. Solve Yes or No question
5. Structured ouput

##### Technique

1. CoT prompting
2. A few shots prompting

##### Target LLMs:
1. meta-llama/Llama-3.2-1B-Instruct 
2. meta-llama/Meta-Llama-3-8B-Instruct
3. ContactDoctor/Bio-Medical-Llama-3-8B

In [ ]:
import torch
import os
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [2]:
def best_dtype():
    """Return the best dtype for the device"""
    if torch.cuda.is_available():
        if torch.cuda.is_bf16_supported():
            return torch.bfloat16
        else:
            return torch.float16
        
    return torch.float32

def best_device():
    """Return the device type"""
    return "cuda" if torch.cuda.is_available() else "cpu"

def login_huggingface():
    """Login HaggingFace"""
    load_dotenv()
    login(os.getenv("HUGGINGFACE_KEY"))
    print("Login HuggingFace!")  

In [3]:
login_huggingface()

Login HuggingFace!


##### Task1: Reasoning
##### Prompts:

In [4]:
prompt_last_letters_basic = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise but brief problem-solver. "
     "Explain your reasoning in a few short steps labeled 'Step 1:', 'Step 2:', 'Step 3:' etc., "
     "then end with a single line 'Result: <value>'. Avoid long explanations."),
    ("human",
     "Given three English words: {w1}, {w2}, {w3}.\n"
     "Task:\n"
     "- For each word, ignore trailing spaces/punctuation and find the last alphabetic letter (A–Z/a–z).\n"
     "- Concatenate these three letters in the SAME order (word1 → word2 → word3).\n"
     "- For example, you got 'how', 'are', 'you', each word's last letter are 'w', 'e', 'u'. Output: 'weu'  .\n"
     "Output:\n"
     "- Write step-by-step text (Step 1/2/3...).\n"
     "- End with one final line: Result: <concatenated_string>.")
])

prompt_simple_math = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise but brief math tutor. "
     "Explain in short steps labeled 'Step 1:', 'Step 2:', ... "
     "then finish with a single line 'Result: <number>'."),
    ("human",
     "John has 5 apples. His father has 8 apples. He gives 2 apples to his father. "
     "How many apples does his father have?\n"
     "Solve step by step and end with: Result: <number>.")
])

prompt_moderate_math = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise but brief math tutor. "
     "Explain in short steps labeled 'Step 1:', 'Step 2:', ... "
     "then finish with a single line 'Result: <number>'."),
    ("human",
     "John has 12 apples. His father has 9 apples. John gives 3 apples to his father. "
     "Then John's father gives one-third (1/3) of his apples to John's sister. "
     "John's sister eats 2 apples and returns the rest to John. "
     "How many apples does John have at the end?\n"
     "Solve step by step and end with: Result: <number>.")
])

##### Task2: Generate questions based on given content
##### Prompts:

In [5]:
prompt_generate_questions = ChatPromptTemplate.from_messages([
    ("system",
     "You are a cautious medical student who generates clinically relevant, self-contained questions. "
     "Ground every question strictly in the provided document; do not invent details or rely on outside knowledge."),
    ("user",
     "Instructions:\n"
     "1) From the document below, write exactly 3 unique QUESTIONS in English only.\n"
     "2) Cover different medical perspectives (aim for breadth), such as: symptoms/signs; diagnosis/differential; "
     "investigations/labs/imaging; treatment/procedures; medications (dose, interactions, adverse effects); "
     "contraindications/precautions; risk factors/prognosis; prevention/patient counseling; special populations "
     "(e.g., pregnancy, breastfeeding, pediatrics, geriatrics).\n"
     "3) Each question must be concise (≤ 25 words), self-contained (avoid pronouns like 'it/this/that'), and "
     "directly supported by the document.\n"
     "4) Do NOT provide any answers or explanations.\n"
     "5) Return ONLY a JSON list that matches exactly this schema: [\"question1\", \"question2\", ..., \"question3\"]\n"
     "6) Enclose the JSON list between <json> and </json> tags.\n"
     "7) If the document does not support 3 distinct perspectives, still produce 3 questions but avoid near-duplicates; "
     "prefer covering as many perspectives as possible."),
    ("user", "Document:\n{doc}\n")
])

contents = [
    "phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine",
    "Sinner's physical struggles appeared to begin late in the second set, and he rushed to place ice towels around his neck at the changeover before the start of the third. In the decider, the Italian was limping between points and frequently massaged his right thigh. At the 2-1 changeover, he didn't sit and instead put his legs up on his bench to try and ward off cramp.",
    "Artificial intelligence (AI) investing is still the dominant theme in the stock market. This checks out, as it's where a massive amount of capital is getting invested to build out computing infrastructure and train models. Although it seems like AI has been the prevailing market theme for some time, there are multiple indications that this will persist for several more years, making AI a great place to invest today.",
    "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine",
    "if you are taking scheduled doses of aluminum hydroxide, magnesium hydroxide, take the missed dose as soon as you remember it. however, if it is almost time for the next dose, skip the missed dose and continue your regular dosing schedule. do not take a double dose to make up for a missed one.about Aluminum Hydroxide and Magnesium Hydroxide"
]

##### Task 3: Answer question based on given content
##### Prompts:

In [ ]:
prompt_answer_questions = PromptTemplate(
    template="""You are a helpful assistant that answers questions **only** using the retrieved documents.

    ## Instructions
    - Carefully read the retrieved documents below.
    - Write your answer in a **natural, conversational tone** as if explaining to a person.
    - Your answer must be **strictly based on the retrieved documents** — do not invent or assume information.
    - Do **not** mention or reference the documents explicitly (e.g., do not say "according to the document" or "based on the source").
    - If the documents do not provide enough information, respond with:
      "I'm sorry, the provided documents do not contain enough information to answer this question."

    ## Inputs
    Retrieved Documents:
    {document}

    User Query:
    {question}

    ## Output
    Provide a **clear, natural, and human-like** answer grounded only in the retrieved documents.
    Avoid formal or robotic phrasing, and never cite or reference the documents directly.
    """,
    input_variables=["document", "question"],
)

contents = [
    "phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine",
    "Sinner's physical struggles appeared to begin late in the second set, and he rushed to place ice towels around his neck at the changeover before the start of the third. In the decider, the Italian was limping between points and frequently massaged his right thigh. At the 2-1 changeover, he didn't sit and instead put his legs up on his bench to try and ward off cramp.",
    "Artificial intelligence (AI) investing is still the dominant theme in the stock market. This checks out, as it's where a massive amount of capital is getting invested to build out computing infrastructure and train models. Although it seems like AI has been the prevailing market theme for some time, there are multiple indications that this will persist for several more years, making AI a great place to invest today.",
    "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine",
    "if you are taking scheduled doses of aluminum hydroxide, magnesium hydroxide, take the missed dose as soon as you remember it. however, if it is almost time for the next dose, skip the missed dose and continue your regular dosing schedule. do not take a double dose to make up for a missed one.about Aluminum Hydroxide and Magnesium Hydroxide"
]

questions = [
    "What is phenylephrine used for and how does it work to relieve nasal discomfort?",
    "What physical issues did Sinner experience during the match and how did he try to manage them?",
    "Why is artificial intelligence considered a strong investment theme in the stock market today?",
    "What precautions should be followed when giving phenylephrine to children or taking multiple cold medications that contain it?",
    "What should you do if you miss a scheduled dose of aluminum hydroxide and magnesium hydroxide?"
]


##### Task 4: Solve Yes or No question
##### Prompts:

In [7]:
prompt_self_contained = PromptTemplate(
        template="""You are a grader for a question.
    You must decide whether the question is self-contained—meaning that it is clear, meaningful, and understandable on its own, without any conversation history or external context.
    Here is the user's question: {question} \n
    Return a binary judgment as a JSON object with a single key "score".
    Respond only with {{"score": "yes"}} if the question is self-contained,
    or {{"score": "no"}} if it is not. Do not include any explanation or extra text.""",
        input_variables=["question"],
)

self_contained_questions = [
    {"question":"What is the tallest mountain in South America?", "expect":"""{"score": "yes"}"""},   
    {"question":"Can you think about it?", "expect":"""{"score": "no"}"""},   
    {"question":"Can you explain how blockchain technology works?", "expect":"""{"score": "yes"}"""},  
    {"question":"Do you have a medicine to relieve sinus congestion and pressure?", "expect":"""{"score": "yes"}"""},        
    {"question":"How can I take it?", "expect":"""{"score": "no"}"""},          
    {"question": "Who discovered penicillin?", "expect": """{"score": "yes"}"""},  
    {"question": "Can you tell me more about it?", "expect": """{"score": "no"}"""},  
    {"question": "What are the side effects of ibuprofen?", "expect": """{"score": "yes"}"""},  
    {"question": "Why do you think so?", "expect": """{"score": "no"}"""},  
    {"question": "What is the capital city of Japan?", "expect": """{"score": "yes"}"""},  
]

In [ ]:
prompt_related_question = PromptTemplate(
     template="""You are a conversation coherence grader.
     Your task is to decide whether the user's latest message is logically and topically connected to the previous conversation.
     Conversation history:
     {document}
     User's latest message:
     {question}
     Return only a JSON object with a single key "score":
     - {{"score": "yes"}} if the latest message is coherent and contextually related to the conversation history.
     - {{"score": "no"}} if it is not related or breaks the context.
     No explanation or extra text.""",
     input_variables=["document", "question"],
)  

documents = [
   """AI: phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed.\n 
            HUMAN: What form does Phenylephrine come? """,
   """Sinner's physical struggles appeared to begin late in the second set, and he rushed to place ice towels around his neck at the changeover""",
            
   """AI: amoxicillin is available as a capsule, tablet, chewable tablet, and liquid suspension to take by mouth.\n 
      HUMAN: What forms does amoxicillin come in?""",
   """AI: metformin is usually taken once or twice daily with meals to reduce stomach upset.\n 
      HUMAN: When should I take metformin?""",
   """AI: ibuprofen works by reducing hormones that cause inflammation and pain in the body.\n 
      HUMAN: How does ibuprofen work?""",
   """AI: loratadine may cause headache, dry mouth, or drowsiness in some people.\n 
      HUMAN: What are common side effects of loratadine?""",
   """AI: acetaminophen is generally used to relieve mild to moderate pain and reduce fever.\n 
      HUMAN: What is acetaminophen used for?""",
   """AI: azithromycin is typically taken once daily for 3 to 5 days, depending on the infection type.\n 
      HUMAN: How long do I need to take azithromycin?""",
   """AI: omeprazole should be taken before meals, usually in the morning, to reduce stomach acid production.\n 
      HUMAN: When is the best time to take omeprazole?""",
   """AI: insulin helps lower blood sugar by allowing glucose to enter cells and be used for energy.\n 
      HUMAN: What does insulin do in the body?"""
]  

related_questions = [
   "how can I take it?",
   "what did happen to John?",
   "how can I take it?",
   "how can I take azithromycin?",
   "how can I take it?",
   "how can I take it?",
   "how can I take azithromycin?",
   "how can I take it?",
   "how can I take it?",
   "how can I take it?",
]

expecting = [
   """{"score": "yes"}""", 
   """{"score": "no"}""",
   """{"score": "yes"}""", 
   """{"score": "no"}""", 
   """{"score": "yes"}""", 
   """{"score": "yes"}""", 
   """{"score": "no"}""", 
   """{"score": "yes"}""", 
   """{"score": "yes"}""", 
   """{"score": "yes"}""", 
]

##### Task 5: Structured output

In [9]:
# Pure structure output
prompt_structured_output = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise assistant. "
     "Read the user's request and produce your answer as structured JSON. "
     "Do not include extra explanations or text outside the JSON."),
    ("human",
     "Extract the following information from this sentence:\n"
     "'Alice bought 3 apples for $5 on Monday.'\n\n"
     "Return the result in this JSON format:\n"
     "{{\n"
     '  "person": "",\n'
     '  "item": "",\n'
     '  "quantity": ,\n'
     '  "price": "",\n'
     '  "day": ""\n'
     "}}")
])

##### First of all, Testing meta-llama/Llama-3.2-1B-Instruct

In [26]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )

print(f"Load {model_id} done!")

Load meta-llama/Llama-3.2-1B-Instruct done!


In [28]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

# Wrapper normal piple with huggingfacepipeline
hug_pipe = HuggingFacePipeline(pipeline=pipe)

# Transfer the pipeline to Chat mode.
# Because this is a way we can use ChatPromptTemplate to make better prompt.
Llama_1b = ChatHuggingFace(llm=hug_pipe)

Device set to use cuda


In [ ]:
# Reasoning test
reason_letters_chain = prompt_last_letters_basic | Llama_1b
result = reason_letters_chain.invoke({"w1":"good", "w2":"morning", "w3": "Medipal"})
print("Expecting: dgl\n")
print(result)
# Expecting: 'dgl'
# Result is 'DGGL'
# It is ok!

Expecting: dgl

content="Step 1: \nRemove trailing spaces from 'good' to get 'good'.\nStep 2: \nFind last alphabetic letter in 'good' which is 'D'.\nStep 3: \nFind last alphabetic letter in'morning' which is 'G'.\nStep 4: \nFind last alphabetic letter in 'Medipal' which is 'L'.\nStep 5: \nConcatenate these three letters in the same order to get 'DGGL'." additional_kwargs={} response_metadata={} id='run--c224fad4-a593-4f4a-8632-e769d2fa7a7b-0'


In [ ]:
# simple math
reason_math_chain = prompt_simple_math | Llama_1b
result = reason_math_chain.invoke({})
print(result)

# Expecting: 10
# Result is 3
# bad performance!

content='Step 1: Start with the initial number of apples John has.\nJohn has 5 apples.\n\nStep 2: Calculate the number of apples John gives to his father.\nJohn gives 2 apples to his father.\n\nStep 3: Subtract the number of apples John gave to his father from the initial number of apples John has.\n5 - 2 = 3\n\nResult: 3' additional_kwargs={} response_metadata={} id='run--f64dffbd-9538-4ed5-b943-2a46489568cb-0'


In [ ]:
# a little harder math
reason_math_moderate_chain = prompt_moderate_math | Llama_1b
result = reason_math_moderate_chain.invoke({})
print(result)
# Expecting: 11
# Result is 6
# bad performance again!

content="Step 1: John starts with 12 apples.\nStep 2: John gives 3 apples to his father, leaving him with 12 - 3 = 9 apples.\nStep 3: John's father gives 1/3 of his 9 apples to John, which is (1/3) * 9 = 3 apples. Now John has 9 - 3 = 6 apples.\nStep 4: John's sister eats 2 apples, leaving John with 6 - 2 = 4 apples.\nStep 5: John's sister returns 2 apples to John, making the total number of apples John has 4 + 2 = 6 apples.\n\nResult: 6" additional_kwargs={} response_metadata={} id='run--910fb982-5f8a-4801-a808-6a5c51722c1b-0'


In [ ]:
# Generate questions based on content
generate_question_chain = prompt_generate_questions | Llama_1b

for c in contents:
    result = generate_question_chain.invoke({"doc": c})
    print(result)

# The generated questions have low quality and the format is a mess.

content='Here are three questions based on the provided information:\n\n<json>\n"question1", "question2", "question3"\n"Is phenylephrine used to treat colds, allergies, and hay fever?", "Does phenylephrine relieve sinus congestion and pressure?", "Is phenylephrine a class of medications called nasal decongestants?"\n</json>\n\n"question4", "question5", "question6"\n"Does phenylephrine relieve nasal discomfort caused by colds, allergies, and hay fever?", "Does phenylephrine reduce swelling of the blood vessels in the nasal passages?", "Is phenylephrine used to relieve sinus pressure?"\n</json>\n\n"question7", "question8", "question9"\n"Is phenylephrine used to relieve nasal discomfort caused by colds, allergies, and hay fever?", "Is phenylephrine used to relieve sinus congestion?", "Is phenylephrine used to relieve nasal pressure?"\n</json>' additional_kwargs={} response_metadata={} id='run--7c33200c-72df-45cd-b410-e4cc40cbaf49-0'
content='<json>\n    "question1": "What is the likely di

In [ ]:
# Answer questions based on content

answer_question_chain = prompt_answer_questions | Llama_1b

for q, c in zip(questions,contents):
    result = answer_question_chain.invoke({"document": c, "question": q})
    print(result)

# not bad!

content="Phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. It's also used to relieve sinus congestion and pressure. To do this, phenylephrine works by reducing swelling of the blood vessels in the nasal passages. This helps to relieve nasal discomfort and congestion." additional_kwargs={} response_metadata={} id='run--6544dea7-458a-4dd6-a7e4-cb91afedbfb2-0'
content="Sinner's physical struggles seemed to start in the second set. He was limping around the court and massaging his right thigh in an attempt to alleviate the discomfort. When the Italian made the 2-1 changeover, he put his legs up on the bench to try and prevent cramps." additional_kwargs={} response_metadata={} id='run--1dce1962-00ea-4842-a2b3-8556eaad2182-0'
content="Artificial intelligence is considered a strong investment theme in the stock market for several reasons. One of the main reasons is that it's where a massive amount of capital is getting invested to build out computing

In [ ]:
# Solve Yes or No question， structured_output
self_contained_chain = prompt_self_contained | Llama_1b
for q in self_contained_questions:
    result = self_contained_chain.invoke({"question": q["question"]})
    print(f"""Expecting:{q["expect"]}, actual result:{result}""")

#it can not handle yes or no question at all!

Expecting:{"score": "yes"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--13fd686f-7e9c-4a88-8603-d662d8ab2ca5-0'
Expecting:{"score": "no"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--a4b889f1-632d-46ae-9b8f-db85d9291776-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--f8bce220-7728-494e-ae32-292f8a859450-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--713c188b-5aa0-4426-b72f-e99432a34ce4-0'
Expecting:{"score": "no"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--0538ac44-d930-45d8-b155-fe15c8cfe4f3-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--7fcddd22-a9dd-46be-bd2d-ca27d2d25ac5-0'
Expecting:{"score": "no"}, actual result:content

In [ ]:
related_chain = prompt_related_question | Llama_1b
for q, d, e in zip(related_questions, documents, expecting):
    result  = related_chain.invoke({"question": q, "document": d})
    print(f"""Expecting:{e}, actual result: {result} """)

#1 out of 10. it can not handle yes or no question at all!

Expecting:{"score": "yes"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--f82fe778-b8f1-41c8-823c-307e729dfdb1-0' 
Expecting:{"score": "no"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--b1094875-b0f9-4041-8ed1-ea9fdd5e1d58-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--57d554ae-899c-4eef-8dd9-22ea6087360b-0' 
Expecting:{"score": "no"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--2e51e744-63d2-4166-9d1a-729867c4689a-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--a183cefd-6b6a-4314-8268-7f906d8c2ba8-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--63be0bf9-031d-4dc4-8110-dfb454270036-0' 
Expecting:{"score": "no"}, actual re

In [ ]:
structured_output_chain = prompt_structured_output | Llama_1b
result  = structured_output_chain.invoke({})
print(f"""Result: {result} """)

# Expecting {\n  "person": "Alice ",\n  "item": "apples",\n  "quantity": 3,\n  "price": "5",\n  "day": "Monday"\n}
# It is OK, but not accuracy

Result: content='{\n  "person": "",\n  "item": "apples",\n  "quantity": 3,\n  "price": "$5",\n  "day": "Monday"\n}' additional_kwargs={} response_metadata={} id='run--45caa7c9-4848-4bfd-814e-a133f5360225-0' 


##### Testing meta-llama/Meta-Llama-3-8B-Instruct

In [18]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )

print(f"Load {model_id} done!")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Load meta-llama/Meta-Llama-3-8B-Instruct done!


In [21]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

# Wrapper normal piple with huggingfacepipeline
hug_pipe = HuggingFacePipeline(pipeline=pipe)

# Transfer the pipeline to Chat mode.
# Because this is a way we can use ChatPromptTemplate to make better prompt.
Llama_8b = ChatHuggingFace(llm=hug_pipe)

Device set to use cuda


In [ ]:
# Reasoning test
reason_letters_chain = prompt_last_letters_basic | Llama_8b
result = reason_letters_chain.invoke({"w1":"good", "w2":"morning", "w3": "Medipal"})
print(result)
# Expecting: 'dgl'
# Result is 'dgl'
# It is pretty good!

content='Here are the steps to solve the problem:\n\nStep 1: Remove trailing spaces and punctuation from each word.\ngood -> good, morning -> morning, Medipal -> Medipal\n\nStep 2: Find the last alphabetic letter (A-Z/a-z) of each word.\ngood -> d, morning -> g, Medipal -> L\n\nStep 3: Concatenate these letters in the same order (word1 → word2 → word3).\nd-g-L\n\nResult: dgl' additional_kwargs={} response_metadata={} id='run--e1852036-12c9-4dda-9575-6da38c4f4b10-0'


In [ ]:
reason_math_chain = prompt_simple_math | Llama_8b
result = reason_math_chain.invoke({})
print(result)
# Expecting: 10
# Result is 10
# It is pretty good!

content="Step 1: John's father initially has 8 apples.\nStep 2: John gives 2 apples to his father, so the father receives 2 more apples.\nStep 3: To find the total number of apples his father has now, add the initial 8 apples to the 2 apples received: 8 + 2 = 10\nResult: 10" additional_kwargs={} response_metadata={} id='run--cfa477ae-9516-4d08-823f-21b69ead357d-0'


In [ ]:
# a little harder math
reason_math_moderate_chain = prompt_moderate_math | Llama_8b
result = reason_math_moderate_chain.invoke({})
print(result)
# Expecting: 11
# Result is 11
# Amazing job!

content="Here's the solution:\n\nStep 1: John has 12 apples initially.\nStep 2: John gives 3 apples to his father, so John has 12 - 3 = 9 apples left.\nStep 3: John's father has 9 apples initially, and then receives 3 apples from John, so John's father has 9 + 3 = 12 apples.\nStep 4: John's father gives 1/3 of his apples to John's sister, which is 12 ÷ 3 = 4 apples.\nStep 5: John's sister eats 2 apples, so she has 4 - 2 = 2 apples left.\nStep 6: John's sister returns the remaining 2 apples to John.\nStep 7: John already had 9 apples, and now receives 2 more from his sister, so John has 9 + 2 = 11 apples.\n\nResult: 11" additional_kwargs={} response_metadata={} id='run--8360f818-175b-4275-a164-ead553316eae-0'


In [ ]:
# Generate questions based on content
generate_question_chain = prompt_generate_questions | Llama_8b

for c in contents:
    result = generate_question_chain.invoke({"doc": c})
    print(result)

# Pretty good!

content='<json>\n["What are the common uses of phenylephrine?", "Can phenylephrine treat the underlying cause of nasal discomfort?", "What is the mechanism of action of phenylephrine as a nasal decongestant?"]\n</json>' additional_kwargs={} response_metadata={} id='run--ef813a3f-c710-4f41-b9b3-7a6998f2a8aa-0'
content='<json>\n["What are the initial symptoms exhibited by Sinner during the second set?", "What is the potential cause of Sinner\'s limping and thigh massaging in the decider?", "What is Sinner\'s attempt to alleviate his cramp symptoms at the 2-1 changeover?"]\n</json>' additional_kwargs={} response_metadata={} id='run--ac0d01d6-b3ba-4bb3-bcf8-19071b901edd-0'
content='<json>\n["What are the key areas of computing infrastructure being built to support AI investing?", "What are the indications that AI will remain a prevailing market theme for several more years?", "What is the expected impact of AI investing on the stock market in the near future?"]\n</json>' additional_kwargs=

In [ ]:
# Answer questions based on content

answer_question_chain = prompt_answer_questions | Llama_8b

for q, c in zip(questions,contents):
    result = answer_question_chain.invoke({"document": c, "question": q})
    print(result)

# Pretty good!

content="Phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. It's also great for relieving sinus congestion and pressure. When you're feeling stuffy or congested, phenylephrine helps by reducing the swelling of the blood vessels in your nasal passages. This swelling is what's causing your discomfort, so by shrinking it down, phenylephrine helps ease your symptoms. It's like a quick fix to help you breathe easier and feel more comfortable." additional_kwargs={} response_metadata={} id='run--8d4a3c09-73d9-49bd-b99d-7d5e2df39e4f-0'
content='It seems like Sinner was dealing with some physical struggles during the match. He started to show signs of distress towards the end of the second set, and it only got worse from there. He had to put ice towels around his neck at the changeover before the third set, which suggests he was experiencing some kind of discomfort or pain. Later on, he was limping between points and massaging his right thigh, which imp

In [ ]:
# Solve Yes or No question， structured_output
self_contained_chain = prompt_self_contained | Llama_8b
for q in self_contained_questions:
    result = self_contained_chain.invoke({"question": q["question"]})
    print(f"""Expecting:{q["expect"]}, actual result:{result}""")

# 10 out of 10

Expecting:{"score": "yes"}, actual result:content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--7a7660c2-e76b-4dad-96ba-1d710374f8bb-0'
Expecting:{"score": "no"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--e9816c7c-1ce0-4b22-b2f8-e9c6524cab80-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--7754d14f-57f2-473b-abf3-823c69640242-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--8a07e2aa-9402-4df4-81a9-b2bb131524d2-0'
Expecting:{"score": "no"}, actual result:content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--82701dd2-6e18-46b9-a7e5-b10e0fca5370-0'
Expecting:{"score": "yes"}, actual result:content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--f0c498ef-7abc-42c3-86e3-6ce908791831-0'
Expecting:{"score": "no"}, actual result:con

In [25]:
related_chain = prompt_related_question | Llama_8b
for q, d, e in zip(related_questions, documents, expecting):
    result  = related_chain.invoke({"question": q, "document": d})
    print(f"""Expecting:{e}, actual result: {result} """)

# 9 out of 10

Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--88978225-0c7d-4ca8-8263-658261c431e4-0' 
Expecting:{"score": "no"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--b33a768c-9550-4617-9ce9-c63f7684d9e3-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--8582f436-a0fe-4a9a-973a-3d3d3b1d3da3-0' 


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Expecting:{"score": "no"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--4ce5db25-5b2f-45c5-b650-226114c0ca24-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--43ce4240-bc03-462c-884f-1129a5821849-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--cee784fd-0c13-4a09-8d96-3cc807cb03d4-0' 
Expecting:{"score": "no"}, actual result: content='{"score": "no"}' additional_kwargs={} response_metadata={} id='run--5d5cf5fb-7585-4fdb-b8ca-efddc86587d8-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--32d9d25f-0e1f-42d8-813f-b8e9b02fdf29-0' 
Expecting:{"score": "yes"}, actual result: content='{"score": "yes"}' additional_kwargs={} response_metadata={} id='run--8b62e857-b408-4f10-a05d-b0ea16c5cbc1-0' 
Expecting:{"score": "yes"}, actu

In [ ]:
structured_output_chain = prompt_structured_output | Llama_8b
result  = structured_output_chain.invoke({})
print(f"""Result: {result} """)

# Expecting {\n  "person": "Alice ",\n  "item": "apples",\n  "quantity": 3,\n  "price": "5",\n  "day": "Monday"\n}
# Amazing job!

Result: content='{\n  "person": "Alice",\n  "item": "apples",\n  "quantity": 3,\n  "price": 5,\n  "day": "Monday"\n}' additional_kwargs={} response_metadata={} id='run--5283cdc7-1d15-44d8-8a45-a94768beefa4-0' 


##### Testing ContactDoctor/Bio-Medical-Llama-3-8B

In [10]:
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype = best_dtype(),
            device_map={"":best_device()}, 
            low_cpu_mem_usage=True           
        )

print(f"Load {model_id} done!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load ContactDoctor/Bio-Medical-Llama-3-8B done!


In [13]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_full_text=False,   
    )

# Wrapper normal piple with huggingfacepipeline
hug_pipe = HuggingFacePipeline(pipeline=pipe)

# Transfer the pipeline to Chat mode.
# Because this is a way we can use ChatPromptTemplate to make better prompt.
Medical_Llama_8b = ChatHuggingFace(llm=hug_pipe)

Device set to use cuda


In [ ]:
# Reasoning test
reason_letters_chain = prompt_last_letters_basic | Medical_Llama_8b
result = reason_letters_chain.invoke({"w1":"good", "w2":"morning", "w3": "Medipal"})
print(result)

# Expecting: 'dgl'
# Result is 'mg'
# It is pretty bad!

content=' Step 1: Remove trailing spaces and punctuation from each word.\nStep 2: Find the last alphabetic letter (A–Z/a–z) of each word.\nStep 3: Concatenate the three letters in the same order.\nResult: mg' additional_kwargs={} response_metadata={} id='run--d0d49b2a-03bc-46b7-a210-a4f48e085686-0'


In [ ]:
reason_math_chain = prompt_simple_math | Medical_Llama_8b
result = reason_math_chain.invoke({})
print(result)
# Expecting: 10
# Result is 10
# It is pretty good!

content=" Step 1: Initial number of apples father has: 8\nStep 2: Apples John gives to father: 2\nStep 3: Subtract apples given from father's initial apples: 8 - 2 = 6\nResult: 6" additional_kwargs={} response_metadata={} id='run--53f40231-ee84-4a4e-b7a3-19884634620a-0'


In [15]:
# a little harder math
reason_math_moderate_chain = prompt_moderate_math | Medical_Llama_8b
result = reason_math_moderate_chain.invoke({})
print(result)
# Expecting: 11
# Result is 13
# The step is wrong, so result is wrong!

content=" Step 1: John has 12 apples initially and gives 3 to his father, so he has 12 - 3 = 9 apples left.\nStep 2: John's father now has 9 + 3 = 12 apples and gives one-third of 12 apples to John, so John gets 12/3 = 4 apples from his father.\nStep 3: John now has 9 + 4 = 13 apples and his sister eats 2 apples from John, so John has 13 - 2 = 11 apples left.\nStep 4: John's sister had 4 apples and ate 2, so she has 4 - 2 = 2 apples left and returns the rest to John, which is 2 apples.\nStep 5: John now has 11 + 2 = 13 apples.\nResult: 13" additional_kwargs={} response_metadata={} id='run--7a275202-b5ae-4cd5-a8b2-b9ba3e4ffa04-0'


In [16]:
# Generate questions based on content
generate_question_chain = prompt_generate_questions | Medical_Llama_8b

for c in contents:
    result = generate_question_chain.invoke({"doc": c})
    print(result)

# questions are good. but format is not stable.

content=' <json>\n["Does phenylephrine relieve nasal discomfort caused by a common cold?", "Can phenylephrine be used to relieve sinus congestion?", "Does phenylephrine belong to a class of medications called nasal decongestants?"\n</json>' additional_kwargs={} response_metadata={} id='run--76b62a93-03db-4dfc-bbb2-4c503e63a206-0'
content=' <json>\n["Why would Sinner use ice towels around his neck at a changeover?", "What might have contributed to Sinner\'s physical struggles in the second set?", "What might be the purpose of Sinner frequently massaging his right thigh?"\n</json>' additional_kwargs={} response_metadata={} id='run--df5f178c-93e0-4cea-9837-c11cd674ce17-0'
content=' <json>\n["What are the key drivers of the current dominance of AI investing in the stock market?", "How does the massive capital investment in AI computing infrastructure and model training contribute to its ongoing prevalence in the market?", "What are the likely implications of continued large-scale investmen

In [ ]:
# Answer questions based on content

answer_question_chain = prompt_answer_questions | Medical_Llama_8b

for q, c in zip(questions,contents):
    result = answer_question_chain.invoke({"document": c, "question": q})
    print(result)

# Pretty good!

content=' Phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. It works by reducing swelling of the blood vessels in the nasal passages.' additional_kwargs={} response_metadata={} id='run--3e7be86d-9503-4b18-b7a4-33a28ed21a8d-0'
content=' Sinner experienced physical struggles, particularly in the second set. He applied ice towels around his neck at the changeover before the start of the third set, indicating he might have had heat-related issues. During the third set, he was limping between points and frequently massaged his right thigh, suggesting he had some kind of muscle or joint pain. At the 2-1 changeover in the decider, instead of sitting down like usual, he put his legs up on his bench to try and manage what seemed to be cramps.' additional_kwargs={} response_metadata={} id='run--78ab53e3-d3ce-4ac5-b00d-9d24364f779f-0'
content=' Artificial intelligence (AI) is considered a strong investment theme in the stock market today because a massiv

: 

In [19]:
# Solve Yes or No question， structured_output
self_contained_chain = prompt_self_contained | Medical_Llama_8b
for q in self_contained_questions:
    result = self_contained_chain.invoke({"question": q["question"]})
    print(f"""Expecting:{q["expect"]}, actual result:{result}""")

# 8 out of 10

Expecting:{"score": "yes"}, actual result:content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--ceba652a-2b20-47cd-b5a2-8851ba2a7586-0'
Expecting:{"score": "no"}, actual result:content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--2175daff-bff4-4b41-bd7a-1cd1f0accd5b-0'
Expecting:{"score": "yes"}, actual result:content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--6d2b4316-1b60-4004-9c5f-cc1bae9e03c6-0'
Expecting:{"score": "yes"}, actual result:content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--95f56aa3-560e-45a4-a96c-5f860bf12b1f-0'
Expecting:{"score": "no"}, actual result:content=' {"score": "no"}' additional_kwargs={} response_metadata={} id='run--b53857be-4e84-4996-83ee-9730b7fba5a0-0'
Expecting:{"score": "yes"}, actual result:content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--aa12bf99-f67f-446f-9c40-b8790d253d1c-0'
Expecting:{"score": "no"}, actual res

In [21]:
related_chain = prompt_related_question | Medical_Llama_8b
for q, d, e in zip(related_questions, documents, expecting):
    result  = related_chain.invoke({"question": q, "document": d})
    print(f"""Expecting:{e}, actual result: {result} """)

# 10 out of 10

Expecting:{"score": "yes"}, actual result: content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--2f00db09-ddcc-4df6-b6ab-300f78bb1cc1-0' 
Expecting:{"score": "no"}, actual result: content=' {"score": "no"}' additional_kwargs={} response_metadata={} id='run--5eb100f8-7315-4647-a71c-eee9b781a3f3-0' 
Expecting:{"score": "yes"}, actual result: content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--55888080-db4a-4ca8-ad0d-fb79b8a1512b-0' 
Expecting:{"score": "no"}, actual result: content=' {"score": "no"}' additional_kwargs={} response_metadata={} id='run--31afe703-ee01-43cc-be57-d0e5e4256065-0' 
Expecting:{"score": "yes"}, actual result: content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--ec95d1f5-6ec0-47e7-b336-45b223c85315-0' 
Expecting:{"score": "yes"}, actual result: content=' {"score": "yes"}' additional_kwargs={} response_metadata={} id='run--2a000c19-155a-48c7-9bff-6a5df3e92ca5-0' 
Expecting:{"score": "no"},

In [21]:
structured_output_chain = prompt_structured_output | Medical_Llama_8b
result  = structured_output_chain.invoke({})
print(f"""Result: {result} """)

# Expecting {\n  "person": "Alice ",\n  "item": "apples",\n  "quantity": 3,\n  "price": "5",\n  "day": "Monday"\n}
# It is pretty good!

Result: content=' {\n  "person": "Alice",\n  "item": "apples",\n  "quantity": 3,\n  "price": 5,\n  "day": "Monday"\n}' additional_kwargs={} response_metadata={} id='run--e228831b-023b-4f5f-a97a-401b93cde14a-0' 
